In [3]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
from PIL import Image, ImageTk
from scipy.ndimage import gaussian_filter
from skimage.draw import disk
from skimage.measure import label, regionprops_table
from skimage.color import label2rgb
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

def generate_varied_fluorescent_yeast_image(dimensions, num_cells, avg_diameter, var_diameter, noise_level):
    # Initialize an empty RGB image
    image = np.zeros((dimensions[0], dimensions[1], 3), dtype=np.float32)

    # Generate random cell properties
    cell_radii = np.random.normal(loc=avg_diameter / 2, scale=var_diameter / 2, size=num_cells)
    intensities = np.random.normal(loc=220, scale=20, size=num_cells)

    for _ in range(num_cells):
        x, y = np.random.randint(0, dimensions[0], size=2)  # Random position for each cell
        radius = max(1, np.abs(cell_radii[_]))  # Ensure radius is positive and at least 1
        rr, cc = disk((x, y), radius, shape=image.shape[:2])
        intensity = max(100, min(255, intensities[_]))  # Clamp intensity to valid range
        image[rr, cc, 0] = intensity  # High intensity in red channel
        image[rr, cc, 1] = intensity * 0.8  # Slightly lower intensity in green channel
        image[rr, cc, 2] = 0  # No intensity in blue channel

    # Apply Gaussian smoothing and add background noise
    image = gaussian_filter(image, sigma=1)
    background_noise = noise_level * np.random.normal(size=image.shape[:2])
    image[:, :, 0] += background_noise
    image[:, :, 1] += background_noise

    return np.clip(image, 0, 255).astype(np.uint16)  # Convert to uint16 for grayscale representation

def segment_and_analyze(image):
    # Convert image to grayscale for segmentation
    grayscale_image = np.mean(image[:, :, :3], axis=2).astype(np.uint8)
    
    # Simple thresholding to segment the cells
    thresh = grayscale_image > np.percentile(grayscale_image, 50)  # threshold based on intensity
    labeled_image = label(thresh, connectivity=2)
    
    # Extract properties using regionprops_table
    properties = ['label', 'centroid', 'area', 'orientation', 'major_axis_length',
                  'minor_axis_length', 'eccentricity', 'perimeter', 'mean_intensity']
    cell_info_table = regionprops_table(labeled_image, intensity_image=grayscale_image, properties=properties)
    
    # Convert the dictionary to a pandas DataFrame
    df = pd.DataFrame(cell_info_table)
    df['roundedness'] = 4 * np.pi * df['area'] / (df['perimeter'] ** 2)  # add roundedness calculation
    
    return labeled_image, df

def display_segmented_image(labeled_image):
    # Use a continuous colormap for the labeled image
    colormap = cm.get_cmap('viridis')
    colored_image = label2rgb(labeled_image, bg_label=0, bg_color=(0, 0, 0), colors=[colormap(i / labeled_image.max()) for i in range(labeled_image.max() + 1)])
    return (colored_image * 255).astype(np.uint8)

def generate_and_display_image():
    try:
        # Get parameters from UI entries
        width = int(width_entry.get())
        height = int(height_entry.get())
        num_cells = int(num_cells_entry.get())
        avg_diameter = float(avg_diameter_entry.get())
        var_diameter = float(var_diameter_entry.get())
        noise_level = float(noise_level_entry.get())

        # Generate the yeast cell image
        yeast_image_rgb = generate_varied_fluorescent_yeast_image((width, height), num_cells, avg_diameter, var_diameter, noise_level)

        # Convert to grayscale for display
        yeast_image_gray = np.mean(yeast_image_rgb, axis=2).astype(np.uint8)  # Convert to 8-bit grayscale for display
        original_image_pil = Image.fromarray(yeast_image_gray)

        # Segment and color the yeast image
        labeled_image, cell_properties_df = segment_and_analyze(yeast_image_rgb)
        segmented_image = display_segmented_image(labeled_image)
        segmented_image_pil = Image.fromarray(segmented_image)

        # Display the original and segmented images in Tkinter
        root.after(0, lambda: update_image_display(original_image_pil, segmented_image_pil))

    except ValueError as e:
        messagebox.showerror("Invalid input", f"Please check your input values.\nError: {e}")

def update_image_display(original_image_pil, segmented_image_pil):
    # Convert images to Tkinter-compatible format and update the labels
    original_image_tk = ImageTk.PhotoImage(image=original_image_pil)
    segmented_image_tk = ImageTk.PhotoImage(image=segmented_image_pil)

    original_image_label.config(image=original_image_tk)
    original_image_label.image = original_image_tk  # Keep a reference to avoid garbage collection

    segmented_image_label.config(image=segmented_image_tk)
    segmented_image_label.image = segmented_image_tk  # Keep a reference to avoid garbage collection

# Initialize the main Tkinter window
root = tk.Tk()
root.title("Fluorescent Yeast Cell Image Generator and Segmenter")

# Input fields for each parameter
tk.Label(root, text="Image Width:").pack()
width_entry = tk.Entry(root)
width_entry.insert(0, "1024")
width_entry.pack()

tk.Label(root, text="Image Height:").pack()
height_entry = tk.Entry(root)
height_entry.insert(0, "1024")
height_entry.pack()

tk.Label(root, text="Number of Cells:").pack()
num_cells_entry = tk.Entry(root)
num_cells_entry.insert(0, "500")
num_cells_entry.pack()

tk.Label(root, text="Average Diameter of Cells:").pack()
avg_diameter_entry = tk.Entry(root)
avg_diameter_entry.insert(0, "10")
avg_diameter_entry.pack()

tk.Label(root, text="Variance in Diameter:").pack()
var_diameter_entry = tk.Entry(root)
var_diameter_entry.insert(0, "2")
var_diameter_entry.pack()

tk.Label(root, text="Background Noise Level:").pack()
noise_level_entry = tk.Entry(root)
noise_level_entry.insert(0, "0.01")
noise_level_entry.pack()

# Button to generate the image
generate_button = tk.Button(root, text="Generate and Segment Image", command=generate_and_display_image)
generate_button.pack(pady=10)

# Labels to display the original and segmented images side by side
original_image_label = tk.Label(root)
original_image_label.pack(side="left", padx=10, pady=10)

segmented_image_label = tk.Label(root)
segmented_image_label.pack(side="right", padx=10, pady=10)

# Start the Tkinter main loop
root.mainloop()


/var/folders/1p/vtxv8w395h148f3j5c16srhh0000gr/T/ipykernel_26076/1256312310.py:59: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('viridis')
